### Setup

In [1]:
## path
path = '/home/andrschl/Documents/projects/Zindi_AutomaticSpeechRecognition/'
# path = 'drive/MyDrive/Colab Notebooks/'
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# installing
!pip install datasets
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qk5gt11a
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-qk5gt11a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-py3-none-any.whl size=2120912 sha256=488699b4a27bcd10edc8f860f9ca7c5780c5033d4a1ae3d025d54e06ac7f3242
  Stored in directory: /tmp/pip-ephem-wheel-cache-3l07apdx/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers


In [3]:
## load packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os.path

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F

import librosa as lb

from transformers import Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor

from sklearn.model_selection import train_test_split

In [4]:
# seeding
random.seed(10)
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed_all(10)

In [5]:
torch.cuda.is_available()

True

### Load Data

In [6]:
# check out dataframe
df = pd.read_csv(path + 'data/ASR_train.csv')
print(df.keys())
print(df.head(5))

Index(['ID', 'up_votes', 'down_votes', 'age', 'gender', 'transcription'], dtype='object')
                                                  ID  up_votes  down_votes  \
0  002e50c29ac2890c7cb3b3d63dcbe512bc6850de206ca8...         2           0   
1  0031672b4484f963c8a07babe6f713dd559539d44140e8...         2           0   
2  00362ccc6b48d3ea225e12ddf8a06a9d582cccc03b0353...         2           0   
3  0042cdb4d4a015cddacc26d88faffdd714b7a27213d2b3...         2           0   
4  00439f02fa8f0dade934bdb317199b71662e9396f6bb81...         2           0   

        age  gender                  transcription  
0       NaN     NaN                        Malicka  
1  twenties  female  Ecole Elementaire Pikine Nord  
2  twenties    male         Cimetière de Cambérène  
3  twenties    male                Tournalou Boune  
4  twenties    male             Pharmacie Golf Sud  


In [7]:
# suppress annoying warnings while reading audio files
import warnings
warnings.filterwarnings('ignore')

In [8]:
## read train set into memory & store as .feather
nsamples = len(df)

# check if already existent
if os.path.isfile(path + 'data/ASR_train_audio'+str(nsamples)+'.ft'):
    print ("File exist")
    df = pd.read_feather(path + 'data/ASR_train_audio'+str(nsamples)+'.ft')
else:
    print("File does not exist")

    # initialize with list
    audio_signals = len(df['ID'])*[[0]]
    df['audio_signal'] = audio_signals

    # functional but not elegant (nor fast probably)
    for k in range(nsamples):
        id = df.iloc[k]['ID']
        path_data = os.path.join(path + 'data/clips/', id+'.mp3')
        waveform, rate = lb.load(path_data, sr=16*1e3)
        df.at[k, 'audio_signal'] = waveform

        if k % 100 == 0:
            print('file '+ str(k))

    # store as faster feather format
    df[:nsamples].to_feather(path + 'data/ASR_train_audio'+str(nsamples)+'.ft')

    #
    df = df[:nsamples]

File exist


In [9]:
## read test set into memory & store as .feather
df_test = pd.read_csv(path + 'data/ASR_test.csv')
nsamples = len(df_test)

# check if already existent
if os.path.isfile(path + 'data/ASR_test_audio'+str(nsamples)+'.ft'):
    print ("File exist")
    df_test = pd.read_feather(path + 'data/ASR_test_audio'+str(nsamples)+'.ft')
else:
    print("File does not exist")

    # initialize with list
    audio_signals = len(df_test['ID'])*[[0]]
    df_test['audio_signal'] = audio_signals

    # functional but not elegant (nor fast probably)
    for k in range(nsamples):
        id = df_test.iloc[k]['ID']
        path_data = os.path.join(path + 'data/clips/', id+'.mp3')
        waveform, rate = lb.load(path_data, sr=16*1e3)
        df_test.at[k, 'audio_signal'] = waveform

        if k % 100 == 0:
            print('file '+ str(k))

    # store as faster feather format
    df_test[:nsamples].to_feather(path + 'data/ASR_test_audio'+str(nsamples)+'.ft')

    #
    df_test = df_test[:nsamples]

File exist


In [10]:
## train valid split
# df_train -> used in optimization
# df_valid -> used to evaluate model during optimization
# df_valid2 -> independent set for testing
df_train, df_valid2 = train_test_split(df, test_size=0.2, random_state=1234)
df_train, df_valid = train_test_split(df_train, test_size=0.2, random_state=1234)

In [11]:
## dataset library (1-2GB/s data processing)
from datasets import Dataset

data_train = Dataset.from_pandas(df_train[['ID', 'transcription', 'audio_signal']])
data_valid = Dataset.from_pandas(df_valid[['ID', 'transcription', 'audio_signal']])

### Pre-Processing

In [12]:
## Lower casing (no punctuation included)
import re
chars_to_ignore= '[\,\?\.\!\;\:\"\“\%\”\�]'

def remove_special_characters(batch):
#     batch['text'] = batch["transcription"].lower() + ' ' # lower casing + word separator at the end
    batch['text'] = re.sub(chars_to_ignore, '', batch["transcription"]).lower() + ' '
    return batch

data_train = data_train.map(remove_special_characters, remove_columns=['transcription'], num_proc=4)
data_valid = data_valid.map(remove_special_characters, remove_columns=['transcription'], num_proc=4)

In [13]:
## build vocabulary
# Q: exclude {, | -} ?
'''
def extract_all_chars(batch):
    all_text = " ".join(batch['text'])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = data_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data_train.column_names)
vocab_valid = data_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data_train.column_names)
'''

'\ndef extract_all_chars(batch):\n  all_text = " ".join(batch[\'text\'])\n  vocab = list(set(all_text))\n  return {"vocab": [vocab], "all_text": [all_text]}\n\nvocab_train = data_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data_train.column_names)\nvocab_valid = data_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data_train.column_names)\n'

In [14]:
'''
## creating vocab dictionary
import json
import os

#
vocab_list = list(set(vocab_train["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

# add unknown token, blank token
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

print(len(vocab_dict)) # dim required for linear layer

vocab_path = os.path.join(path, "data/vocab.json")
with open(vocab_path, 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
'''

'\n## creating vocab dictionary\nimport json\nimport os\n\n#\nvocab_list = list(set(vocab_train["vocab"][0]))\n\nvocab_dict = {v: k for k, v in enumerate(vocab_list)}\nprint(vocab_dict)\n\n# add unknown token, blank token\nvocab_dict["[UNK]"] = len(vocab_dict)\nvocab_dict["[PAD]"] = len(vocab_dict)\n\nprint(len(vocab_dict)) # dim required for linear layer\n\nvocab_path = os.path.join(path, "data/vocab.json")\nwith open(vocab_path, \'w\') as vocab_file:\n    json.dump(vocab_dict, vocab_file)\n'

In [17]:
## tokenizer (for output text)
# tokenizer = Wav2Vec2CTCTokenizer(vocab_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token=" ")
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-xlsr-53-french") # IMPORTANT: before used Wav2VecTokenizer (not CTC)

#tokenizer.get_vocab()

In [18]:
## feature extractor (best guess: for input to cut into windows, normalize etc.)
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

## processor (combine tokenizer and feature extractor)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [19]:
## extract input_values (normalization)
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["audio_signal"], sampling_rate=16*1e3).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

# parameter num_proc does not exist in the currently used version of datasets
data_train = data_train.map(prepare_dataset, remove_columns=data_train.column_names, batch_size=8,  batched=True, num_proc=4)
data_valid = data_valid.map(prepare_dataset, remove_columns=data_valid.column_names, batch_size=8, batched=True, num_proc=4)

### Training

In [ ]:
!pip install wandb
import wandb

import datetime
now = datetime.datetime.now()
now = now.strftime("%d-%m-%Y_%H:%M")

wandb.init(project="ASR_Wolof",
           entity="andrschl",
           name = now,
           group="Wav2Vec2.0_XLSR_large_french")

In [20]:
## data collator (dynamic padding)
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        # input_values, attention_mask, labels
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
## metric
!pip install jiwer
from datasets import load_metric

wer_metric = load_metric("wer")

def compute_metrics(pred):
    # argmax of softmax
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # -100 id -> pad token
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    # prediction id -> character
    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics?
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
## model
## Note: play around with hyperparameters (take training to laptop and perform grid search?)

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53-french",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, # save GPU memory
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id, # define pad token
    #vocab_size=len(processor.tokenizer) -> mis-match of last layer due to vocab size
)

# TODO: remove output layer and replace with vocabulary sized

In [ ]:
model.to('cuda');

In [ ]:
## Freeze CNN layers (no fine tuning as stated in paper)
model.freeze_feature_extractor()

In [ ]:
# # TESTING: freeze all layers

# for name, param in model.named_parameters():
#     # param.requires_grad = False
#     if 'lm_head' not in name:
#     param.requires_grad = False

#     if param.requires_grad:
#     print(name)


In [ ]:
## hyperparameters
from transformers import TrainingArguments

path_model = os.path.join(path, '/model/wav2vec2-large-xlsr-french_'+now)

training_args = TrainingArguments(
    output_dir=path + '/model/wav2vec2-large-xlsr-french_'+now,
    group_by_length=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=70,
    learning_rate=3e-4,
    warmup_steps=20,
    save_total_limit=1,
    report_to="wandb"
)

In [ ]:
## Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_train,
    eval_dataset=data_valid,
    tokenizer=processor.feature_extractor,
)

In [ ]:
## start training
model.train()
trainer.train()
wandb.finish()

### Evaluation

In [32]:
from datasets import load_metric

#from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
if not 'model' in globals():
    print('Load model')
    model = Wav2Vec2ForCTC.from_pretrained(path + '/model/wav2vec2-large-xlsr-french-test/checkpoint-1650/').to("cuda")

model.eval();

# new processor if not yet existent
if not 'processor' in globals():
    print('Load processor')
    tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-xlsr-53-french")
    feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
    processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
    # processor = Wav2Vec2Processor.from_pretrained('./drive/MyDrive/Colab Notebooks/model/wav2vec2-large-xlsr-french-test/checkpoint-150/') # for some reason the file is not found

def prepare_dataset(batch):
    return processor(batch, return_tensors="pt", sampling_rate=16000)

wer_ = []

#
input_dict = df_valid2['audio_signal'].apply(prepare_dataset)
label_str = ''
pred_str = ''

for idx in range(len(df_valid2)):
    #print('-----------------')
    logits = model(input_dict.values[idx].input_values.to("cuda")).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]

    '''
    pred = []
    for word in processor.decode(pred_ids).split(' '):
    sim = difflib.get_close_matches(word, xv, n=1)
    if sim == []:
      sim = word
    else:
      sim = sim[0]

    pred.append(sim)

    #print("Prediction:")
    pred_str+= ' '.join(pred)+ ' '
    '''
    pred_str+= processor.decode(pred_ids)+ ' '

    #print("\nReference:")
    label_str+= df_valid2["transcription"].values[idx].lower()+ ' '

    # need same length for wer_metric
    #label_str = label_str.ljust(len(pred_str))
    #pred_str = pred_str.ljust(len(label_str))
wer_.append(wer_metric.compute(predictions=[pred_str], references=[label_str]))

np.mean(wer_)

0.8111085071478791

In [33]:
wer_metric.compute(predictions=[pred_str], references=[label_str])

0.8111085071478791

In [34]:
audio_signal = df[df['ID']=='de242ae13e8bc5c14e01295dd52eb0d3cc5b7e224507a6decd664eecdad6de5d9fb81c592fae6aadf4830955c36e9620cf07ea38b07675e513b23c0842233596']['audio_signal'].values
input_dict = processor(audio_signal[0], return_tensors="pt", sampling_rate=16*1e3)

logits = model(input_dict.input_values.to("cuda")).logits
pred_ids = torch.argmax(logits, dim=-1)[0]

#print("Prediction:")
pred_str = processor.decode(pred_ids)
print(pred_str)

rue baffassene


### Prediction

In [35]:
## prediction
# load data (dataframe) -> empty entries
df_test = pd.read_feather(path+'/data/ASR_test_audio1564.ft')
df_test = df_test[['ID', 'audio_signal']]

df_test.head()

'''
# pre-process (processor)
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-xlsr-53-french") #
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# pass through model
modelTest = Wav2Vec2ForCTC.from_pretrained('./drive/MyDrive/Colab Notebooks/model/wav2vec2-large-xlsr-french-test/checkpoint-280/').to("cuda")
'''

def prepare_dataset(batch):
    return processor(batch, return_tensors="pt", sampling_rate=16000)

In [36]:
# run through processor
import difflib
input_dict = df_test['audio_signal'].apply(prepare_dataset)
preds = []

# run through model and decoder
for i in range(len(df_test)):
  logits = model(input_dict[i].input_values.to('cuda')).logits
  pred_ids = torch.argmax(logits, dim=-1)
  preds.append(processor.decode(pred_ids[0]))

  '''
  pred = []
  for word in processor.decode(pred_ids[0]).split(' '):
    sim = difflib.get_close_matches(word, xv, n=1)
    if sim == []:
      sim = word
    else:
      sim = sim[0]
        
    pred.append(sim)
  
  preds.append(''.join(pred))
  '''


# save as csv
dfpred = pd.DataFrame(list(zip(list(df_test['ID'].values), preds)), columns=['ID', 'transcription'])
dfpred['transcription'] = dfpred['transcription'].replace([''], ' ') # replace empty prediction with " "
dfpred.to_csv(path+'/predictions/prediction_'+now+'.csv', index=False)

### Word model/ Nearest neighbor

In [85]:
from nltk.probability import FreqDist
from wordcloud import WordCloud, ImageColorGenerator

#
words = df['transcription']
allwords = []

for wordlist in words:
  allwords += list(wordlist.lower().split())

# histogram
mostcommon_small = FreqDist(allwords).most_common(10000)
xv, yv = zip(*mostcommon_small)

'25-04-2021_22:13'

In [89]:
from nltk.probability import FreqDist
from wordcloud import WordCloud, ImageColorGenerator

#
words = dfpred['transcription']
allwords = []

for wordlist in words:
  allwords += list(wordlist.split())

# histogram
mostcommon_small = FreqDist(allwords).most_common(10000)
xp, yp = zip(*mostcommon_small)

fig, ax = plt.subplots(nrows=2, figsize=(22,10))
ax[0].bar(xv,yv)
ax[1].bar(xp,yp)
plt.xlabel('Words')
plt.ylabel('Frequency of Words')
plt.xticks(rotation=60)
plt.show()

0.42934282848412836

In [100]:
count_all = 0
count_notshared = 0
count = 0
for k, word in enumerate(xp):
  count_all += yp[k]
  if word not in xv:
    count_notshared += yp[k]
    #print(word+ ' '+ str(yp[k]))
    count += 1

print(count_all)
print(count_notshared)
print(count)

' '

0                       affricatell avées
1       nanlayydie fon guerdem taliboubes
2                         afficatell abes
3                       moskue de camerin
4                 cité safkot tivoion pol
                      ...                
1559              bankooff affrica pikine
1560                          tigo almadi
1561                          wigi gramba
1562              pharmacie rokayya wakam
1563                       hora la banque
Name: transcription, Length: 1564, dtype: object

,ID,transcription
0,00416cff4f818d3dfd99c9178ff0e268e7575500c8baa5...,affricatell avées
1,00891ba561e80e135f9d12b9fa1347f0a2560998f7ea16...,nanlayydie fon guerdem taliboubes
2,00a508027ed4edf0bd3db79f45f4ed6e1b89fba6482c10...,afficatell abes
3,00ac13cd0d93e35c1ff672cc106ad94d1ea9b93fcf049a...,moskue de camerin
4,00c2d5baf4719bf01b990a8924e99bda043cd462147193...,cité safkot tivoion pol
...,...,...
1559,ff0da457e7a3986035995912803e42261c5f5f448c126b...,bankooff affrica pikine
1560,ff1808218a15fa576c405314e4de4bda56c44f849ff1b5...,tigo almadi
1561,ff5b9a45d60600e875e0a031b1d7076c9cbdeb1c48c09c...,wigi gramba
1562,ff98e108ec61d3bd485734b83f21be77820549dab1cac1...,pharmacie rokayya wakam
